<a href="https://colab.research.google.com/github/cmhrabi/AgenticDesignPatterns/blob/main/notebooks/Chapter_1_Prompt_Chaining_(Code_Example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install langchain langchain-community langchain-openai langgraph

In [26]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# For better security, load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv('/content/.env')
# Make sure your OPENAI_API_KEY is set in the .env file

# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
    model="anthropic/claude-sonnet-4",
    temperature=0
    )

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 3: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
    "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# -- Promp 3: Create a performace rating based on the specifications
prompt_rating = ChatPromptTemplate.from_template(
    "Give a performance rating based on the specification of the computer: \n\n{json_specs}"
)

# -- Prompt 4: Transform back to JSON
propmt_transform_rating = ChatPromptTemplate.from_template(
    "Transform back into JSON object with 'cpu', 'memory', 'storage', and 'performance_score' (score from 1-10) as keys: \n\n{json_specs}\n\n{performance_rating}"
)

# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()

# The full chain passes the output of the extraction chain into the 'specifications'
# variable for the transformation prompt.
spec_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)

rating_chain = (
    {"json_specs": spec_chain}
    | prompt_rating
    | llm
    | StrOutputParser()
)

full_chain = (
    {"json_specs": spec_chain, "performance_rating": rating_chain}
    | propmt_transform_rating
    | llm
    | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 1.5 GHz octa-core processor, 8GB of RAM, and a 500mb NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n-- Final JSON Output --")
print(final_result)


-- Final JSON Output --
```json
{
  "cpu": "1.5 GHz octa-core processor",
  "memory": "8GB",
  "storage": "500GB NVMe SSD",
  "performance_score": 6.5
}
```
